# <font color='green'>Lending Club Loan Data Analysis</font>

### DESCRIPTION

Create a model that predicts whether or not a loan will be default using the historical data.

### Problem Statement:  

For companies like Lending Club correctly predicting whether or not a loan will be a default is very important. In this project, using the historical data from 2007 to 2015, you have to build a deep learning model to predict the chance of default for future loans. As you will see later this dataset is highly imbalanced and includes a lot of features that make this problem more challenging.

**Domain:** Finance

Analysis to be done: Perform data preprocessing and build a deep learning prediction model. 

**Content: **

Dataset columns and definition:

 

**credit.policy:** 1 if the customer meets the credit underwriting criteria of LendingClub.com, and 0 otherwise.

**purpose:** The purpose of the loan (takes values "credit_card", "debt_consolidation", "educational", "major_purchase", "small_business", and "all_other").

**int.rate:** The interest rate of the loan, as a proportion (a rate of 11% would be stored as 0.11). Borrowers judged by LendingClub.com to be more risky are assigned higher interest rates.

**installment:** The monthly installments owed by the borrower if the loan is funded.

**log.annual.inc:** The natural log of the self-reported annual income of the borrower.

**dti:** The debt-to-income ratio of the borrower (amount of debt divided by annual income).

**fico:** The FICO credit score of the borrower.

**days.with.cr.line:** The number of days the borrower has had a credit line.

**revol.bal:1** The borrower's revolving balance (amount unpaid at the end of the credit card billing cycle).

**revol.util:** The borrower's revolving line utilization rate (the amount of the credit line used relative to total credit available).

**inq.last.6mths:** The borrower's number of inquiries by creditors in the last 6 months.

**delinq.2yrs:** The number of times the borrower had been 30+ days past due on a payment in the past 2 years.

**pub.rec:** The borrower's number of derogatory public records (bankruptcy filings, tax liens, or judgments).

 

**Steps to perform:**

Perform exploratory data analysis and feature engineering and then apply feature engineering. Follow up with a deep learning model to predict whether or not the loan will be default using the historical data.

**Tasks:**

1.     Feature Transformation

    * Transform categorical values into numerical values (discrete)

2.     Exploratory data analysis of different factors of the dataset.

3.     Additional Feature Engineering

      * You will check the correlation between features and will drop those features which have a strong correlation

    * This will help reduce the number of features and will leave you with the most relevant features

4.     Modeling

    * After applying EDA and feature engineering, you are now ready to build the predictive models

    * In this part, you will create a deep learning model using Keras with Tensorflow backend



In [ ]:
# Import all Important Librarys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
data=pd.read_csv("loan_data.csv")

In [ ]:
data.head(10)

In [ ]:
#this data set has 9578 rows and 14 columns
data.shape

In [ ]:
data.describe()

In [ ]:
# check null values
data.isnull().sum()

In [ ]:
data.isnull().sum().any()

In [ ]:
data.dtypes

Exploratory data analysis of different factors of the dataset.

In [ ]:
data['not.fully.paid'].value_counts()

0 - Full Paid

1 - Not Paid

imbalanced data

In [ ]:
sns.countplot(x=data['not.fully.paid'])
plt.show()

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot(x=data['purpose'])
plt.show()

In [ ]:
# purpose and not fully paid
plt.figure(figsize=(15,6))
sns.countplot(x='purpose',hue='not.fully.paid',data=data)
plt.show()

In [ ]:
# bi variate analysis
sns.jointplot(x='fico',y='int.rate',data=data, kind='hex', color='g')
plt.show()

In [ ]:
sns.scatterplot(x='fico',y='int.rate',data=data)
plt.show()

In [ ]:
sns.histplot(data['fico'])
plt.show()

In [ ]:
sns.histplot(x='fico', hue='not.fully.paid', data=data)
plt.show()

#### Feature Transformation
#### Transform categorical values into numerical values (discrete)

In [ ]:
# Handle imbalanced dataset
data['not.fully.paid'].value_counts()

In [ ]:
not_fully_paid_0 = data[data['not.fully.paid']==0]
not_fully_paid_1 = data[data['not.fully.paid']==1]

In [ ]:
not_fully_paid_0.shape

In [ ]:
not_fully_paid_1.shape

In [ ]:
#resample
data_minor_upsample=resample(not_fully_paid_1,replace=True,n_samples=8045)

In [ ]:
new_data=pd.concat([not_fully_paid_0,data_minor_upsample])

In [ ]:
#shuffle
new_data=shuffle(new_data)

In [ ]:
new_data['not.fully.paid'].value_counts()

In [ ]:
new_data.shape

In [ ]:
new_data.dtypes

In [ ]:
# convert purpose into num data
le = LabelEncoder()

In [ ]:
for i in new_data.columns:
    if new_data[i].dtypes =='object':
        new_data[i]=le.fit_transform(new_data[i])

In [ ]:
new_data.dtypes

### Additional Feature Engineering

#### You will check the correlation between features and will drop those features which have a strong correlation

#### This will help reduce the number of features and will leave you with the most relevant features

In [ ]:
new_data.corr()

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(new_data.corr(),annot=True)

In [ ]:
#see the sorted results
new_data.corr().abs()['not.fully.paid'].sort_values(ascending=False)

In [ ]:
new_data.columns

In [ ]:
# take columns
X=new_data[['credit.policy','purpose', 'int.rate', 'installment','fico','revol.bal','revol.util','inq.last.6mths','pub.rec']]

In [ ]:
X.shape

In [ ]:
y=new_data['not.fully.paid']

In [ ]:
# Create train set & test set
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
# Apply scaling
sc=StandardScaler()

In [ ]:
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

#### In this part, you will create a deep learning model using Keras with Tensorflow backend


In [ ]:
# create the architecture
# 2 ANN layer
model=Sequential()
model.add(Dense(19,activation='relu',input_shape=[9]))
model.add(Dropout(0.20))

model.add(Dense(10,activation='relu'))
model.add(Dropout(0.20))

# output layer
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
# compile the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
early_stop=EarlyStopping(monitor='val_loss',min_delta=0.001,mode='min',patience=10,verbose=1)

In [ ]:
model.fit(X_train,y_train,
          epochs=50,
          batch_size=256,
          validation_data=(X_test,y_test),
          callbacks=[early_stop])

In [ ]:
history=model.fit(X_train,y_train,
          epochs=50,
          batch_size=256,
          validation_data=(X_test,y_test))

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred

In [ ]:
predictions=(y_pred>0.5).astype('int')

In [ ]:
predictions

In [ ]:
y_test

In [ ]:
accuracy_score(predictions,y_test)

In [ ]:
print(classification_report(predictions,y_test))

In [ ]:
model.save('loan_default1.h5')

Model2 Architecture

In [ ]:
# create the architecture model2
from tensorflow.keras.layers import BatchNormalization
# batch Normalization
# First ANN layer
model1=Sequential()
model1.add(Dense(128,activation='relu',input_shape=[9]))
model1.add(BatchNormalization())
model1.add(Dropout(0.20))

# Second ANN layer
model1.add(Dense(64,activation='tanh'))
model1.add(BatchNormalization())
model1.add(Dropout(0.20))
           

# third ANN layer
model1.add(Dense(32,activation='relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.20))

# output layer
model1.add(Dense(1,activation='sigmoid'))

In [ ]:
model1.summary()

In [ ]:
# compile the model
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history=model1.fit(X_train,y_train,
          epochs=100,
          batch_size=256,
          validation_data=(X_test,y_test))

In [ ]:
model1.evaluate(X_test,y_test)

In [ ]:
model1.evaluate(X_train,y_train)

Hyparameter tuning in Keras

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
def build_model(hp):
    model=Sequential()
    
    # first hidden layer
    model.add(Dense(units=hp.Int('units',min_value=32,max_value=1024,step=16),
                   activation=hp.Choice('activation',['relu','tanh']),input_shape=[9]))
    
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('rate',min_value=0.1,max_value=0.5,step=0.1)))
                      
        
    # Second hidden layer
    model.add(Dense(units=hp.Int('units',min_value=32,max_value=1024,step=16),
                   activation=hp.Choice('activation',['relu','tanh'])))
    
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('rate',min_value=0.1,max_value=0.5,step=0.1)))
    
     # third hidden layer
    model.add(Dense(units=hp.Int('units',min_value=32,max_value=1024,step=16),
                   activation=hp.Choice('activation',['relu','tanh'])))
    
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('rate',min_value=0.1,max_value=0.5,step=0.1)))
    
    model.add(Dense(1,activation='sigmoid'))
    
    learning_rate=hp.Float('learning_rate',min_value=0.001,max_value=0.1,step=0.01)
        
    model.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate),
                 metrics=['accuracy'])
    return model

In [ ]:
import keras_tuner as kt

In [ ]:
build_model(kt.HyperParameters())

In [ ]:
rtuner=kt.RandomSearch(hypermodel=build_model,
                       objective='val_accuracy',
                       max_trials=10                   
                      )

In [ ]:
rtuner.search(X_train,y_train,
             epochs=50,validation_data=(X_test,y_test),
             verbose=2)

In [ ]:
par=rtuner.get_best_hyperparameters()

In [ ]:
par

In [ ]:
models=rtuner.get_best_models()

In [ ]:
len(models)

In [ ]:
models[0].summary()

In [ ]:
y_pred=models[0].predict(X_test)>=0.5

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)